# Check dataset

In [1]:
import sys,os,glob,time,copy,warnings,cProfile,shutil,matplotlib
from IPython import display
import h5py as h5
import numpy as np
import scipy.fftpack
import scipy.signal
import scipy.io as io
import skimage.restoration as deconv
from math import cos, sin, radians
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.transforms import offset_copy
import cartopy.crs as ccrs
from cartopy.io.img_tiles import Stamen, OSM
import cartopy.feature as cfeature
from obspy.geodetics.base import gps2dist_azimuth
from obspy.geodetics import locations2degrees
from obspy.taup import TauPyModel
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
matplotlib.rcParams['pdf.fonttype'] = 42


from pyproj import Geod

import m_pycorr.mods.dd as dd
import m_pycorr.mods.lang as lang

import m_manage_data as m_mod

import ipdb

%matplotlib widget

In [2]:
path          = "data_4.0hz/events/"
list_of_files = glob.glob(path + '/*/*.h5')

lat    = []
lon    = []
id_sta = []
ch     = []

for i_file in list_of_files:
#i_file = 'data_4.0hz/events/glob_03/2014-12-09T12-00-00.000000Z_mww_6.0.h5'
    h5f   = h5.File(i_file,'r')
    dic   = m_mod.recursively_load_dic_from_h5(h5f, path='/_metadata/sta/')
    dic_ev= m_mod.recursively_load_dic_from_h5(h5f, path='/_metadata/ev/')

    for net_lvl in dic.keys():
        for sta_lvl in dic[net_lvl]:
            lat.append(dic[net_lvl][sta_lvl]['lat'][()])
            lon.append(dic[net_lvl][sta_lvl]['lon'][()])
            id_sta.append(net_lvl + '/' + sta_lvl)
            isN =  str(int('/' + net_lvl + '/' + sta_lvl + '/N' in h5f))
            isE =  str(int('/' + net_lvl + '/' + sta_lvl + '/E' in h5f))   
            isZ =  str(int('/' + net_lvl + '/' + sta_lvl + '/Z' in h5f)) 
            ch.append( isZ + isN + isE )
    
    h5f.close()
    #dd.dd(dic)

In [5]:
plt.close('all')
m_mod.mmap(111,lon,lat,ch=ch,xylabel=True,res=2)
plt.scatter(dic_ev['lon'], dic_ev['lat'], marker='*', c='g', s=500,transform=ccrs.PlateCarree(),cmap='plasma_r',alpha=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
plt.close('all')

model = TauPyModel(model="PREM")
ray_p_pcp=[]
vect_epi_dist =  np.linspace(0,90,181)
for dist in vect_epi_dist:
    #print(dist)
    arrivals = model.get_travel_times(source_depth_in_km=0,
                                  distance_in_degree=dist,
                                  phase_list=["PcP"])
    arr = arrivals[0]
    ray_p_pcp.append(arr.ray_param)
    
    
ray_p_pcp = np.array(ray_p_pcp)
plt.plot(vect_epi_dist,np.array(ray_p_pcp))

vec_dist  = np.zeros([len(id_sta),1])
vec_az    = np.zeros([len(id_sta),1])
vec_baz   = np.zeros([len(id_sta),1])

for counterA, staA in enumerate(id_sta):
    (vec_dist[counterA],vec_az[counterA],vec_baz[counterA]) = gps2dist_azimuth(dic_ev['lat'], dic_ev['lon'], lat[counterA], lon[counterA])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
plt.close('all')

geoid = Geod(ellps="WGS84")  

ivec1 = vec_dist/1000/111.91 > 130
ivec2 = vec_dist/1000/111.91 < 180
valid_idxA = np.where(ivec1 * ivec2)[0]


#m_mod.mmap(111,np.array(lon)[valid_idxA],np.array(lat)[valid_idxA],np.array(ch)[valid_idxA].tolist(),xylabel=True,res=2)
vect_dist_pcp = []
for idist in vec_dist[valid_idxA]:
    ray_to_find = model.get_travel_times(source_depth_in_km=0,
                                  distance_in_degree=idist/1000/111.91,
                                  phase_list=["PKIKP","PKP"])[0].ray_param
    vect_dist_pcp.append(vect_epi_dist[np.argmin(np.abs(ray_p_pcp-ray_to_find))])

term_lon,term_lat,term_baz = geoid.fwd(np.array(lon)[valid_idxA],np.array(lat)[valid_idxA], vec_baz[valid_idxA]+180.,np.array(vect_dist_pcp)*1000*111.91) 

#plt.plot(vect_dist_pcp)

m_mod.mmap(111,np.array(lon)[valid_idxA],np.array(lat)[valid_idxA],ch=np.array(ch)[valid_idxA].tolist(),xylabel=True,res=2)
plt.scatter(term_lon,term_lat, marker='o', c='g', s=20,transform=ccrs.PlateCarree(),cmap='plasma_r',alpha=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plt.close('all')

mat_dist   = np.zeros([len(id_sta),len(valid_idxA)])
bool_dist  = np.zeros([len(id_sta),len(valid_idxA)])

for counterA, staA in enumerate(np.array(id_sta)[valid_idxA].tolist()):
    #ipdb.set_trace()
    az,baz,mat_dist[:,counterA] = geoid.inv(np.tile(term_lon[counterA],np.array(lon).size), np.tile(term_lat[counterA],np.array(lon).size), lon , lat)
    bool_dist[:,counterA] = (mat_dist[:,counterA]/1000/111.91 < 0.25 * vect_dist_pcp[counterA])
    
plt.imshow(bool_dist,aspect='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plt.close('all')

valid_idxB = np.where(np.sum(bool_dist,1)>0)[0]
valid_idxA_c = valid_idxA[np.where(np.sum(bool_dist,0)>0)[0]]


m_mod.mmap(111,np.array(lon)[valid_idxA_c],np.array(lat)[valid_idxA_c],ch=None,xylabel=True,res=2,proj='azi',ev=[147,-63])

n = 180
col = pl.cm.inferno(np.linspace(0,1,n))

xfil = open('list_xcorr_PcP_TEST.txt', 'w')

dist_pair = []
for idx, counterA in enumerate(valid_idxA.tolist()):
    for counterB in np.where(bool_dist[:,idx])[0]:
        xfil.write(id_sta[counterA].replace('/','.') + '_' + id_sta[counterB].replace('/','.') + '\n')
        dist_pair.append(geoid.line_length([lon[counterA],lon[counterB]], [lat[counterA], lat[counterB]])/1000/111.91)
        plt.plot([lon[counterA],lon[counterB]], [lat[counterA], lat[counterB]], color=col[int(dist_pair[-1])], linewidth=0.5, alpha=0.25, transform=ccrs.Geodetic())

plt.scatter(dic_ev['lon'], dic_ev['lat'], marker='*', c='g', s=500,transform=ccrs.PlateCarree(),cmap='plasma_r',alpha=0.5)
plt.scatter(np.array(lon)[valid_idxB],np.array(lat)[valid_idxB], marker='o', c='darkviolet', s=20,transform=ccrs.PlateCarree(),cmap='plasma_r',alpha=0.5)

xfil.close()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
plt.savefig('map_pcp_paths.pdf')

In [10]:

plt.close('all')
plt.hist(dist_pair,bins=180)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([  3.,   3.,   1.,   2.,   4.,   3.,   4.,   4.,   7.,   6.,   6.,
          4.,   3.,   8.,  10.,   3.,   2.,  10.,  11.,  18.,  15.,  22.,
         29.,  21.,  23.,  14.,  15.,  30.,  28.,  23.,  16.,  16.,  11.,
         18.,  16.,  12.,  17.,  17.,  22.,  15.,  18.,  13.,  18.,  17.,
         29.,  12.,  18.,  13.,  10.,   7.,  25.,   6.,   9.,   7.,   8.,
          7.,   9.,  14.,   4.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,
          0.,   0.,   0.,   0.,   0.,   2.,   4.,   7.,  11.,  14.,  16.,
         20.,  19.,  19.,  17.,  13., 

In [ ]:
dic_ev['lon'], dic_ev['lat']